## Import Modules

In [ ]:
from __future__ import print_function, division

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import defaultdict
from collections import OrderedDict
import re
import string
from guess_language import guess_language
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
set(stopwords.words('english'))

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics import pairwise_distances
from gensim import corpora, models, similarities, matutils
from nltk.stem.porter import *
import corex as ct
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud, STOPWORDS

#Import modules to collect information missing after scraping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import os
import requests
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

requests.__path__

In [ ]:
df = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Data/Data+Transcripts.pkl', 'rb'))

## Preprocessing Comments

In [ ]:
#If more time available, comments could be another good way to analyze topics and importance of videos.

In [ ]:
#Load Comments
d_comments = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Data/Comments.pkl', 'rb'))

In [ ]:
df['Comments_Transcript'] = df['Links'].map(d_comments)
#df['Comments'] = df['Comments'].str.join(' ')

In [ ]:
#Convert empty lists to string so these rows can be taken out
def stringconversion(comments):
    if len(comments) == 0:
        comments = str(comments)
        return comments
    else:
        return comments

In [ ]:
df['Comments_Transcript'] = df['Comments_Transcript'].apply(lambda x: stringconversion(x))

In [ ]:
df_comments = df[df.Comments_Transcript != '[]']

In [ ]:
len(df_comments)

In [ ]:
def englishcomments(comments):
    """
    Filter comments for english language
    """
    comment_list_overall= []
    comment_list_video = []
    for comment in comments:
        comment_list_video = []
        for single_comment in comment:
            if guess_language(single_comment) == 'en':
                comment_list_video.append(single_comment)
        comment_list_overall.append(comment_list_video)
    return comment_list_overall

In [ ]:
df_comments['Comments_Transcript'] = englishcomments(df_comments['Comments_Transcript'])

In [ ]:
#Create new dataframe for Comments with Links as index
df_comments_new = pd.DataFrame(df_comments['Comments_Transcript'], index = df_comments.Links).stack()

In [ ]:
#äremove empty comments to create dataframe with links as index and comment

## Preprocessing

In [ ]:
#Convert Views number to integer
df['Views']=df.Views.astype('str')
df['Views']=df.Views.replace(to_replace='nan', value= 0)
df['Views'] = df.Views.astype('int32')

In [ ]:
def remove_pattern(txt, pattern):
    try:
        r = re.findall(pattern,txt)
        for i in r:
            txt = txt.replace(i,'')
        return txt
    except:
        return txt 

In [ ]:
def remove_punctuation(text):
    text1 = text.replace("'",'')
    clean_text = re.sub('[%s]'%re.escape(string.punctuation),' ',text1)
    return clean_text

In [ ]:
def remove_numbers(text):
    text1 = re.sub('\w*\d\w*', ' ',text)
    return text1

In [ ]:
def lemmatization(lemmatizer, text):
    new_text = []
    bracket = text.split(' ')
    for word in bracket:
        words = lemmatizer.lemmatize(word)
        new_text.append(words)
    new_text = ' '.join(new_text)
    return new_text
        

In [ ]:
#Replace &quot; with quotation marks
count =0
for index, row in df.iterrows():
    count+=1
    print(count)
    title = row.Title
    title=title.replace("&quot;", '"')
    df['Title'].iloc[index]=title

In [ ]:
#Clean Transcript to conduct Topic Modelling afterwards
#Filter for English videos
df['Transcript_model_en'] = df.Transcript[~df.Transcript.apply(lambda x: guess_language(x) != 'en')]
df = df.dropna(subset = ['Transcript_model_en'], inplace = False)

# remove special characters, numbers, punctuations.
df['Transcript_Description'] = df['Transcript_model_en']+' '+df['Description']
df['Transcript_Description'] = df['Transcript_Description'].apply(lambda x: str(x))
df['Transcript_model'] = df['Transcript_model_en'].apply(lambda x: remove_punctuation(x))
df['Transcript_model'] = df['Transcript_model_en'].apply(lambda x: remove_numbers(x))
df['Transcript_Description'] = df['Transcript_Description'].apply(lambda x: remove_punctuation(x))
df['Transcript_Description'] = df['Transcript_Description'].apply(lambda x: remove_numbers(x))


#Convert Transcripts to lower string
df['Transcript_low'] = df.Transcript_model.apply(lambda x: x.lower())
df['Transcript_Description'] = df.Transcript_Description.apply(lambda x: x.lower())
print('Converted string to lowercase')

#Lemmatization
lemmatizer = WordNetLemmatizer()
df['Transcript_lemmatized'] = df.Transcript_low.apply(lambda x: lemmatization(lemmatizer, x))
df['Transcript_Description_lemmatized'] = df.Transcript_Description.apply(lambda x: lemmatization(lemmatizer,x))

#Tokenization
df['Transcript_sent'] = df.Transcript_low.apply(lambda x: list(ngrams(word_tokenize(x),3)))

# remove special characters, numbers, punctuations.
#df['Transcript_model'] = df['Transcript_low'].str.replace("[^a-zA-Z#]", " ")
#df['Transcript_low']= np.vectorize(remove_pattern)(df['Transcripts'], "@[\w]*")

In [ ]:
df['Transcript'].isna().value_counts()

In [ ]:
def slicedate(date):
    index = date.find('T')
    if index != -1:
        date1 = date[:index]
        return date1
    else:
        date1 = 0
    return date1

In [ ]:
def replacenan(dates):
    list_date = []
    for date in dates:
        if re.findall(r'\d+', date) == []:
            date = 0
        list_date.append(date)  
    return list_date

In [ ]:
def convertdates(dates):
    """
    Converting Month strings to a numerical string
    """
    list_dates = []
    months = {
        'January' : 1,
        'February' : 2,
        'March' : 3,
        'April' : 4,
        'May' : 5,
        'June' : 6,
        'July' : 7,
        'August' : 8,
        'September' : 9, 
        'October' : 10,
        'November' : 11,
        'December' : 12
}
    for date in dates:
        if date != 0:
            space_index = date.find(' ')
            month = date[:space_index].strip()
            try: 
                new_month = months[month]
                year = date[space_index:].strip()
                if new_month < 10:
                    date = year+'-0'+str(new_month)+'-01'
                else:
                    date = year+'-'+str(new_month)+'-01'
            except:
                date = 0
        else:
            date = 0
        list_dates.append(date)
    return list_dates
            
                

In [ ]:
#Slice date of Upload Date
df['Upload Date1'] = df['Upload Date'].apply(lambda x: slicedate(x))

In [ ]:
#Creating a timestamp for each tweet in order to be able to make an hourly count
def timestamp(tweet):
    """
    Creating a timestamp for each tweet in order to be able to make an hourly count
    """
    d = defaultdict(str)
    for i in tweet:
        if i != 0:
            y = datetime.datetime.strptime(i, '%Y-%m-%d')
            timestamp = pd.Timestamp(year = y.year, month = y.month, day = y.day) 
        else:
            timestamp =0
        d[i] = timestamp
    return d

In [ ]:
df['Event Date'] = df['Event Date'].apply(lambda x: str(x))
df['Event Date'] = replacenan(df['Event Date'])

#Convert Month into number
df['Event Date1'] = convertdates(df['Event Date'])

#Convert Event Dates to Timestamp
d_timestamp = timestamp(df['Event Date1'])
df['Event Date Numerical'] = df['Event Date1'].map(d_timestamp)

df.drop(labels = 'Event Date1', inplace = True, axis = 1)

In [ ]:
#Convert Upload Date to Timestamps
d_timestamp = timestamp(df['Upload Date1'])
df['Timestamp'] = df['Upload Date1'].map(d_timestamp)
df.drop(labels='Upload Date1', axis = 1, inplace = True)

In [ ]:
def replacezeros(df, column1, column2):
    df1 = df
    for i in range(len(df1)):
        if df1[column1].iloc[i] == 0:
            df1[column1].iloc[i] = df1[column2].iloc[i]
    return df1
            

In [ ]:
df = replacezeros(df,'Event Date Numerical', 'Timestamp')

In [ ]:
#123 views 
#123 length
#120 Description
#120 Author
#136 Profession
#5 Rating
#121Languages
#571 Comments
#207 Event Date


In [ ]:
df.head()

In [ ]:
pkl.dump(df,open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Data/Model_df.pkl', 'wb'))

## Topic Modeling

In [ ]:
stopwords1 = stopwords.words('english')
stopwords1.append('just')
stopwords1.append('like')
stopwords1.append('com')
stopwords1.append('great')
stopwords1.append('might')
stopwords1.append('talk')
stopwords1.append('went')
stopwords1.append('ca')
stopwords1.append('laughter')
stopwords1.append('actually')
stopwords1.append('fact')
stopwords1.append('nan')
stopwords1.append('ok')
stopwords1.append('yeah')
stopwords1.append('quot')
stopwords1.append('wa')
stopwords1.append('ha')
stopwords1.append('see')
stopwords1.append('well')
stopwords1.append('would')
stopwords1.append('thats')
stopwords1.append('thing')
stopwords1.append('look')
stopwords1.append('im')
stopwords1.append('one')
stopwords1.append('get')
stopwords1.append('know')
stopwords1.append('think')
stopwords1.append('two')
stopwords1.append('say')
stopwords1.append('dont')
stopwords1.append('felt')

In [ ]:
len(stopwords1)

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### Latent Semantic Analysis(LSA)

In [ ]:
def lsa(text):
    #Applying Vectorization
    cv = CountVectorizer(stop_words = stopwords1, max_df = 0.5, min_df = 0.05)
    X_cv = cv.fit_transform(text)
    
    lsa = TruncatedSVD(10)
    doc_topic = lsa.fit_transform(X_cv)
    lsa.explained_variance_ratio_
    
    #Show most common words in topics
    display_topics(lsa, cv.get_feature_names(), 10)
    
    return doc_topic

In [ ]:
doc_topic = lsa(df['Transcript_lemmatized'])

### NMF

In [ ]:
### Non-Negative Matrix Factorization (NMF)
def nmf(text):
    #Applying Vectorization
    tfidf = TfidfVectorizer(stop_words = stopwords1, max_df = 0.21, min_df = 0.01)
    X_cv = tfidf.fit_transform(text)
    
    nmf = NMF(32)
    doc_topic = nmf.fit_transform(X_cv)
    #nmf.explained_variance_ratio_
    
    #Show most common words in topics
    display_topics(nmf, tfidf.get_feature_names(), 10)
    
    return doc_topic, nmf, tfidf

In [ ]:
doc_topic, nmf1, tfidf = nmf(df['Transcript_Description_lemmatized'])

In [ ]:
pd.Series(doc_topic.argmax(axis = 1)).value_counts()

In [ ]:
df.Topic.value_counts()

In [ ]:
df.reset_index(inplace = True)

In [ ]:
df['Topic'] = pd.Series(doc_topic.argmax(axis = 1))
labels=['Religion', 'Neuroscience', 'Solar System', 'Music', 'Medical Treatment','Human Biology', 'Economy',
      'Astrophysics', 'Robotics', 'Politics','Oceanography', 'Education','Africa', 'Linguistics','War','Racial Discrimination',
      'Art','Climate Change','Internet','Cancer','Transportation','Architecture/Design','Sex','Family','Artificial Intelligence',
      'E-Sports','Asia','Virology','Children','Ecosystem','Recovery','Criminality']
for i in range(32):
    df['Topic'] = df['Topic'].replace(to_replace = float(i), value = labels[i])

In [ ]:
labels=['Religion', 'Neuroscience', 'Solar System', 'Music', 'Medical Treatment','Human Biology', 'Economy',
      'Astrophysics', 'Robotics', 'Politics','Oceanography', 'Education','Africa', 'Linguistics','War','Racial Discrimination',
      'Art','Climate Change','Internet','Cancer','Transportation','Architecture/Design','Sex','Family','Artificial Intelligence',
      'E-Sports','Asia','Virology','Children','Ecosystem','Recovery','Criminality']
display_topics(nmf1, tfidf.get_feature_names(), 10, topic_names = labels)

In [ ]:
pkl.dump(df,open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Data/df_Topics.pkl','wb'))

In [ ]:
df = pkl.load(open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Data/df_Topics.pkl','rb'))

In [ ]:
df_dates = df[df.Timestamp != 0]
df['Year']=df_dates.Timestamp.apply(lambda x: pd.Timestamp(year=x.year, month=1, day=1))

In [ ]:
df_y=df.groupby(['Year','Topic'])['Topic'].count()

In [ ]:
df_y = df_y.unstack()

In [ ]:
df_y= df_y.replace(to_replace = np.NaN, value = 0)

In [ ]:
df_y.head()

In [ ]:
df_y.to_csv('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Data/Topics_over_time.csv')

In [ ]:
df_y.plot()

### Corex

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords1, token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True, max_df = 0.4, min_df = 0.01)

doc_word = vectorizer.fit_transform(df['Transcript_lemmatized'])
words = list(np.asarray(vectorizer.get_feature_names()))

In [ ]:
topic_model = ct.Corex(n_hidden=9, words=words, seed=10)
topic_model.fit(doc_word, words=words, docs=df.Transcript_lemmatized, 
                anchors=[['education'], 
                         ['politics', 'government'], 
                         ['medicine', 'health'],
                        ['ocean','earth'],
                        ['society'],
                        ['space', 'planet','star'],
                        ['energy', 'solar'],
                        ['country'],
                        ['city', 'urban']],anchor_strength=0.6)

In [ ]:
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

In [ ]:
#Top documents for each model
topic_model.get_top_docs(topic=0, n_docs=2)

## Recommender System

In [ ]:
def report_error(message, value):
    """
    Returns: An error message about the given value.

    This is a function for constructing error messages to be used in assert statements.
    We find that students often introduce bugs into their assert statement messages, and
    do not find them because they are in the habit of not writing tests that violate
    Preconditions.

    The purpose of this function is to give you an easy way of making error messages
    without having to worry about introducing such bugs. Look at the function
    draw_two_lines for the proper way to use it.

    Parameter message: The error message to display
    Precondition: message is a string

    Parameter value: The value that caused the error
    Precondition: NONE (value can be anything)
    """
    return message+': '+repr(value)

In [ ]:
def cleaninput(text):
    """
    Clean text given by user
    """
    assert type(text) == str, report_error('Input is not a string', text)
    
    #Check for english input texxt
    if guess_language(text) != 'en':
        print('Please provide a proper english text as input')
        return
    text_new = remove_punctuation(text)
    text_new = remove_numbers(text_new)
    text_lower = text_new.lower()
    
    #Lemmatize text
    tokens = word_tokenize(text_lower)
    lemmatizer = WordNetLemmatizer()
    tokens_lemmatized = [lemmatization(lemmatizer, i) for i in tokens]
    text = ' '.join(tokens_lemmatized)
    return text
    

In [ ]:
def searchkeywords(df, keyword, sort = 'Views', number = 5):
    assert type(keyword) == str
    df_keywords = df[df.Keywords_string.str.contains(keyword)]
    if len(df_keywords)>0:
        if sort=='Views' or sort=='Relvance':
            df_keywords = df_keywords.sort_values(by= sort, ascending = False)
        elif sort=='Upload Date':
            df_keywords = df_keywords.sort_values(by= 'Timestamp', ascending = False)    
        return df_keywords.iloc[0:number]
    else:
        return 'No Keywords'

In [ ]:
def checkoverlap(text, vectorizer):
    text_list=text.split(' ')
    vectorizer_names =vectorizer.get_feature_names()
    count = 0
    for word in text_list:
        if word in vectorizer_names:
            count+=1
    if count > 0:
        return True
    else:
        print('Please rephrase your input/wording.')
        return False
        

In [ ]:
def recommendation(text, sort = 'Relevance', number = 5):
#Applying Vectorization
    tfidf = TfidfVectorizer(stop_words = stopwords1, max_df = 0.21, min_df = 0.02)
    cv_mat = tfidf.fit_transform(df['Transcript_Description_lemmatized'])
    nmf = NMF(100)
    nmf_mat = nmf.fit_transform(cv_mat)
    dist = pairwise_distances(nmf_mat)
    recc=dist.argsort(axis=1)[:,0:number+1]
    text = text.lower()
    
    link = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    if link != []:
        link = link[0]
        index = link.find('talks')
        if index == -1:
            print('This link is not a valid Ted Talk Link')
            return
        else:
            link = link[index-1:]
            index_link = list(df[df.Links == link].index)
            recc1 = recc[index_link[0]][1:]
            return recc1
    elif len(text.split(' ')) <3:
        recommendations_keywords = searchkeywords(df, keyword = text, sort = sort, number = number)
        if type(recommendations_keywords) != str:
            if len(recommendations_keywords) > 0 and len(recommendations_keywords)<number:
                missing=number-len(recommendations_keywords)
                top_recc = list(recommendations_keywords.index)
                similar_docs=recc[top_recc[0]][1:]
                for k in range(missing):
                    top_recc.append(similar_docs[k])
                return top_recc
        
            elif len(recommendations_keywords)==number:
                return list(recommendations_keywords.index)
        else:
            #cleaned =cleaninput(text)
            if checkoverlap(text,tfidf):
                cv_new = tfidf.transform([text])
            else:
                return
    else:
        #cleaned = cleaninput(text)
        if checkoverlap(text, tfidf):
            cv_new = tfidf.transform([text])
        else:
            return
        
    nmf_new = nmf.transform(cv_new)
    dist = pairwise_distances(nmf_new,nmf_mat)
    recc=dist.argsort(axis=1)[:,0:number]
    return recc

In [ ]:
tfidf = TfidfVectorizer(stop_words = stopwords1, max_df = 0.21, min_df = 0.02)
cv_mat = tfidf.fit_transform(df['Transcript_Description_lemmatized'])
nmf = NMF(100)
nmf_mat = nmf.fit_transform(cv_mat)
dist = pairwise_distances(nmf_mat)

In [ ]:
pkl.dump(nmf_mat,open('~/Desktop/Projects/Project_5/Flask/data/Flask/NMF_Mat.pkl', 'wb'))
pkl.dump(dist,open('~/Desktop/Projects/Project_5/Flask/data/Flask/Pairwise_dist.pkl', 'wb'))
pkl.dump(nmf,open('~/Desktop/Projects/Project_5/Flask/data/Flask/NMF.pkl', 'wb'))
df['Keywords_string']=df.Keywords.apply(lambda x: ' '.join(x))
pkl.dump(df,open('~/Desktop/Projects/Project_5/Flask/data/Flask/df.pkl', 'wb'))
pkl.dump(tfidf,open('~/Desktop/Projects/Project_5/Flask/data/Flask/tfidf.pkl', 'wb'))

In [ ]:
def recommendoutput(recommender, sort ='Relevance'):
    """
    Converts the array given as output from the recommendation function to a
    list of names of most similar articles.
    """
    if recommender.any() ==None:
        return 'No Recommendations'
    else:
        if len(recommender)==1:
            recommender = recommender[0]
        if sort=='Views':
            df_temp=df[df.index.isin(recommender)].sort_values(by='Views', ascending=False)
            recommender=list(df_temp.index)
        elif sort=='Upload Date':
            df_temp=df[df.index.isin(recommender)].sort_values(by='Timestamp', ascending=False)
            recommender=list(df_temp.index)
        name_list = [(df.Title.iloc[i], 'https://www.ted.com'+str(df.Links.iloc[i])) for i in recommender]
        return name_list

In [ ]:
def recommendationsystem(text,sort='Relevance', number = 5):
    recc = recommendation(text, number=number, sort=sort)
    print(recc)
    return recommendoutput(recc, sort=sort)

In [ ]:
df['Keywords_string']=df.Keywords.apply(lambda x: ' '.join(x))

### Word Clouds

In [ ]:
### Generate wordclouds
def wordcloud1(model, feature_names, no_top_words, topic):
    labels=['Religion', 'Neuroscience', 'Solar System', 'Music', 'Medical Treatment','DNA/Genes', 'Economy',
      'Astrophysics', 'Robotics', 'Politics','Oceanography', 'Education','Africa', 'Linguistics','War','Racial Discrimination',
      'Art','Climate Change','Internet','Cancer','Transportation','Architecture/Design','Sex','Family','Artificial Intelligence',
      'E-Sports','Asia','Virology','Children','Ecosystem','Recovery','Criminality']
    index = labels.index(topic)
    words = model.components_[index]
    words1 = (" ".join([feature_names[i] for i in words.argsort()[:-no_top_words - 1:-1]]))
    cloud = WordCloud(background_color = "white", max_words = 200, stopwords = stopwords1)
    cloud.generate(words1)
    return cloud

In [ ]:
cloud = wordcloud1(nmf1, tfidf.get_feature_names(), 200, 'Ecosystem')

In [ ]:
cloud.to_file('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Presentation/Wordclouds/Ecosystem_cloud.png')

In [ ]:
df.Topic.value_counts()

In [ ]:
#Create String date in order to display in flask app
df['Date_string']=df.Timestamp
count =0
for index,row in df.iterrows():
    count+=1
    print(count)
    z=row.Timestamp
    df['Date_string'].iloc[index]= str(z.month)+"/"+str(z.day)+"/"+str(z.year)

In [ ]:
pkl.dump(df,open('/Users/ferdinandwohlenberg/Desktop/Projects/Project_5/Flask/data/df.pkl', 'wb'))